In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter

In [2]:
img3D = np.zeros((30,30,30))
img3D[15,15,15] = 1

In [ ]:
img3D_conv = gaussian_filter(img3D, sigma=(2,2,2))

In [ ]:
def gauss_filter(dataset, std_space, std_time):
    gauss_dataset = gaussian_filter(dataset, sigma=(2,2,2))
    return gauss_dataset